In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pymorphy2
import concurrent.futures
import nlpaug.augmenter.word as naw
import tqdm  # Для прогресс-бара
from transformers import TFXLMRobertaModel, XLMRobertaTokenizer
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
import numpy as np

In [ ]:
# Инициализация лемматизатора
morph = pymorphy2.MorphAnalyzer()

In [ ]:
# Загрузка данных
data = pd.read_csv('emotions.csv')

In [ ]:
# Загрузка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

In [ ]:
# Функция для предобработки текста
def preprocess_text(text):
    text = text.lower()  # Нормализация к нижнему регистру
    text = re.sub(r'\d+', '', text)  # Удаление цифр
    text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации
    words = [word for word in text.split() if word not in stop_words]  # Удаление стоп-слов
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]  # Лемматизация
    return ' '.join(lemmatized_words)

In [ ]:
# Применение предобработки
data['Текст фразы'] = data['Текст фразы'].apply(preprocess_text)

# Проверка результата
print(data.head())

# Сохранение обработанных данных
data.to_csv('processed_emotions.csv', index=False)

In [ ]:
def augment_single_text(text, back_translation_aug):
    try:
        # Агментация текста с использованием перевода
        return back_translation_aug.augment(text)
    except Exception as e:
        print(f"Error augmenting text: {text[:30]}... Error: {e}")
        return text  # Возвращаем исходный текст, если произошла ошибка

In [ ]:
    back_translation_aug = naw.BackTranslationAug(
        from_model_name='Helsinki-NLP/opus-mt-en-ru',
        to_model_name='Helsinki-NLP/opus-mt-ru-en'
    )

    augmented_data = []

    # Обрабатываем данные по частям (например, по 100 строк)
    for i in tqdm.tqdm(range(0, len(data), chunk_size), desc="Processing data in chunks"):
        chunk = data.iloc[i:i+chunk_size]
        # Используем ThreadPoolExecutor для параллельной обработки
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            for augmented_text in executor.map(lambda text: augment_single_text(text, back_translation_aug), chunk['Текст фразы']):
                augmented_data.append(augmented_text)

    data['Augmented'] = augmented_data
    return data

if __name__ == '__main__':
    try:
        # Загружаем обработанные данные
        data = pd.read_csv(r"processed_emotions.csv")
        print(f"Data loaded successfully. First few rows:\n{data.head()}")

        # Применяем агментацию
        new_data = augment_data(data, 'Недовольство', n_syn=10, n_back=5)

        # Выводим результат
        print(new_data.head())

        # Сохраняем результаты в новый файл
        new_data.to_csv(r"augmented_emotions.csv", index=False)

    except Exception as e:
        print(f"Error: {e}")

In [ ]:
# Загружаем оба файла
processed_data = pd.read_csv("emotions.csv")
augmented_data = pd.read_csv("augmented_emotions.csv")

In [ ]:
# Объединяем данные
data = pd.concat([processed_data, augmented_data], ignore_index=True)

In [ ]:
# Создаем аугментатор для замены синонимов
aug = naw.SynonymAug(aug_p=0.2)  # aug_p - вероятность замены синонимов

# Первая аугментация
augmented_texts_1 = [aug.augment(text) for text in data['Текст фразы']]

# Обновляем данные с первой аугментацией
data['Текст фразы'] = augmented_texts_1

# Вторая аугментация на уже аугментированных данных
augmented_texts_2 = [aug.augment(text) for text in data['Текст фразы']]

# Обновляем данные с второй аугментацией
data['Текст фразы'] = augmented_texts_2

# Сохраняем обновленные данные в новый файл
data.to_csv("augmented_data.csv", index=False)

print("Двойная аугментация завершена. Данные сохранены в 'augmented_data.csv'.")


In [ ]:
# Загрузка данных
raw_data_path = "emotions.csv"
augmented_data_path = "augmented_data.csv"

raw_data = pd.read_csv(raw_data_path)
augmented_data = pd.read_csv(augmented_data_path)

In [ ]:
# Преобразование меток эмоций в числовой формат
raw_data['Эмоция'] = raw_data['Эмоция'].astype('category').cat.codes
augmented_data['Эмоция'] = augmented_data['Эмоция'].astype('category').cat.codes


In [ ]:
# Инициализация токенизатора
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# Разделение сырых данных на обучающую и тестовую выборки
X_train_raw, X_test, y_train_raw, y_test = train_test_split(
    raw_data['Текст фразы'], raw_data['Эмоция'], test_size=0.2, random_state=42
)

# Объединение сырых и аугментированных данных для второго этапа
combined_data = pd.concat([raw_data, augmented_data], ignore_index=True)
X_train_combined = combined_data['Текст фразы']
y_train_combined = combined_data['Эмоция']

In [ ]:
# Токенизация данных
def tokenize_texts(texts):
    return tokenizer(
        texts.tolist(),
        max_length=64,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

# Загрузка модели XLM-RoBERTa для получения эмбеддингов
embedding_model = TFXLMRobertaModel.from_pretrained('xlm-roberta-base')

# Заморозка первых шести слоёв модели
for layer in embedding_model.roberta.encoder.layer[:6]:
    layer.trainable = False

# Оптимизация с использованием шедулера
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=100,
    decay_rate=0.96,
    staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
embedding_model.compile(optimizer=optimizer, loss='categorical_crossentropy')


In [ ]:
# Получение эмбеддингов батчами
def get_embeddings_batchwise(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        tokens = tokenize_texts(batch_texts)
        batch_embeddings = embedding_model(tokens)['last_hidden_state'][:, 0, :].numpy()  # Первый токен [CLS]
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# Главная функция
if __name__ == "__main__":
    # Первый этап: обучение на сырых данных
    print("Этап 1: обучение на сырых данных...")
    train_embeddings_raw = get_embeddings_batchwise(X_train_raw)
    test_embeddings = get_embeddings_batchwise(X_test)

    svm = SVC(kernel='linear', C=1, probability=True)
    svm.fit(train_embeddings_raw, y_train_raw)

    # Оценка на тестовой выборке после первого этапа
    svm_preds_raw = svm.predict(test_embeddings)
    accuracy_raw = accuracy_score(y_test, svm_preds_raw)
    print(f"Точность на сырых данных: {accuracy_raw:.4f}")

    # Второй этап: дообучение на объединенных данных
    print("Этап 2: дообучение на объединенных данных...")
    train_embeddings_combined = get_embeddings_batchwise(X_train_combined)
    svm.fit(train_embeddings_combined, y_train_combined)

    # Оценка на тестовой выборке после второго этапа
    svm_preds_combined = svm.predict(test_embeddings)
    accuracy_combined = accuracy_score(y_test, svm_preds_combined)
    precision_combined = precision_score(y_test, svm_preds_combined, average='weighted')
    recall_combined = recall_score(y_test, svm_preds_combined, average='weighted')
    f1_combined = f1_score(y_test, svm_preds_combined, average='weighted')

    # Вывод метрик
    print(f"Точность на объединенных данных: {accuracy_combined:.4f}")
    print(f"Precision: {precision_combined:.4f}")
    print(f"Recall: {recall_combined:.4f}")
    print(f"F1-Score: {f1_combined:.4f}")

    # Сохранение модели SVM
    model_path = "svm_emotion_model.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(svm, f)
    print(f"Модель SVM сохранена в {model_path}")
